# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [43]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import pprint as pp

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [19]:
weather_df = pd.read_csv("weather_df.csv")
weather_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Arraial do Cabo,75,BR,1586637350,88,-22.97,-42.03,22.15,2.10
1,Feicheng,0,CN,1586637764,56,35.26,117.97,6.93,2.96
2,Orlik,100,RU,1586637354,62,52.52,99.83,-3.06,1.67
3,Rikitea,78,PF,1586637346,74,-23.12,-134.97,25.17,0.60
4,Ushuaia,75,AR,1586637164,70,-54.80,-68.30,7.15,12.30


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [70]:
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
coordinates = weather_df[["Lat", "Lng"]]
rating = weather_df["Humidity"].astype(float)
rating
heat_layer = gmaps.heatmap_layer(coordinates, weights = rating, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=10)

fig = gmaps.figure(layout=figure_layout, map_type="HYBRID")  # Assign the marker layer to a variable
markers = gmaps.marker_layer(coordinates)

fig.add_layer(markers)

fig.add_layer(heat_layer)

# # Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

In [25]:
ideal_df = weather_df[(weather_df["Max Temp"] > 20) & (weather_df["Max Temp"] < 30)]
ideal_df = ideal_df[ideal_df["Humidity"] <= 60]
ideal_df = ideal_df[ideal_df["Cloudiness"] < 40]
ideal_df #= ideal_df.dropna()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
30,Saint George,1,US,1586637238,36,37.10,-113.58,20.15,2.10
37,Morón,2,AR,1586637779,53,-34.65,-58.62,24.15,3.10
40,Concepción del Uruguay,0,AR,1586637780,44,-32.48,-58.24,24.59,0.89
45,Mar del Plata,37,AR,1586637197,56,-38.00,-57.56,22.37,5.70


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [57]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
print(f"Reading API from {base_url}")
hotel_df = pd.DataFrame(columns= ["City" , "Hotel Name", "Hotel Lat", "Hotel Lng"])
for index, location in ideal_df.iterrows():
    
    lat = location["Lat"]
    lng = location["Lng"]
    city = location["City"]
    
    params = {
        "location": f"{lat},{lng}",
        "keyword": "Hotel",
        "radius": 10000,
        "key": g_key
    }
    try:
        response_json = requests.get(base_url, params=params).json()
        hotel_df = hotel_df.append({"City": city,
                                    "Hotel Name": response_json['results'][0]['name'], 
                                    "Hotel Lat": response_json['results'][0]['geometry']['location']['lat'],
                                    "Hotel Lng": response_json['results'][0]['geometry']['location']['lng']}, ignore_index=True)
        print(f"added hotel to {city}")
    except: 
        print(f"Could not add hotel in {city}")
        #response_json

Reading API from https://maps.googleapis.com/maps/api/place/nearbysearch/json
added hotel to Saint George
added hotel to Morón
added hotel to Concepción del Uruguay
added hotel to Mar del Plata


In [59]:
ideal_df = ideal_df.merge(hotel_df)
ideal_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name,Hotel Lat,Hotel Lng
0,Saint George,1,US,1586637238,36,37.10,-113.58,20.15,2.10,Red Lion Hotel & Conference Center St. George,37.093510,-113.586869
1,Morón,2,AR,1586637779,53,-34.65,-58.62,24.15,3.10,Holiday Inn Buenos Aires Ezeiza Airport,-34.733796,-58.519908
2,Concepción del Uruguay,0,AR,1586637780,44,-32.48,-58.24,24.59,0.89,Antigua Fonda,-32.487357,-58.233203
3,Mar del Plata,37,AR,1586637197,56,-38.00,-57.56,22.37,5.70,Sheraton Mar del Plata Hotel,-38.031830,-57.539713


In [64]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_df.iterrows()]
locations = ideal_df[["Hotel Lat", "Hotel Lng"]]

['\n<dl>\n<dt>Name</dt><dd>Red Lion Hotel & Conference Center St. George</dd>\n<dt>City</dt><dd>Saint George</dd>\n<dt>Country</dt><dd>US</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Holiday Inn Buenos Aires Ezeiza Airport</dd>\n<dt>City</dt><dd>Morón</dd>\n<dt>Country</dt><dd>AR</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Antigua Fonda</dd>\n<dt>City</dt><dd>Concepción del Uruguay</dd>\n<dt>Country</dt><dd>AR</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Sheraton Mar del Plata Hotel</dd>\n<dt>City</dt><dd>Mar del Plata</dd>\n<dt>Country</dt><dd>AR</dd>\n</dl>\n']

In [78]:
# Add marker layer ontop of heat map
fig = gmaps.figure(layout=figure_layout, map_type="HYBRID")  # Assign the marker layer to a variable
markers = gmaps.marker_layer(locations)
symbol_layer = gmaps.symbol_layer(
            locations, info_box_content=hotel_info)
fig.add_layer(markers)
heat_layer = gmaps.heatmap_layer(coordinates, weights = weather_df["Humidity"], 
                                 dissipating=False, max_intensity=100,
                                 point_radius=10)
fig.add_layer(heat_layer)
fig.add_layer(symbol_layer)

# Display Map
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…